# Exploring use cases & research methods
This demo notebook shows some ways of using `edsl` to conduct research, including data labeling, cognitive testing and creating new methods.

In [1]:
# EDSL should be automatically installed when you run this notebook. If not, run the following command:
# ! pip install edsl

## Cognitive testing
In this example we use the tools to evaluate some draft survey questions and suggest improvements.

In [2]:
from edsl.questions import QuestionFreeText
from edsl import Agent, Scenario, Model

Create a relevant persona and assign it to an agent:

In [3]:
agent_description = "You are an expert in survey methodology and evaluating questionnaires."

agent = Agent(traits = {"background": agent_description})

Identify a set of texts for review (these can also be imported):

In [4]:
draft_texts = [
    "Do you feel the product is almost always of good quality?",
    "On a scale of 1 to 5, where 1 means strongly agree and 5 means strongly disagree, how satisfied are you with our service?",
    "Do you believe our IT team's collaborative synergy effectively optimizes our digital infrastructure?",
    "What do you think of our recent implementation of Project X57?"
]

Construct a question about the texts, which will be added as a parameter of the question individually:

In [5]:
question = QuestionFreeText(
    question_name = "review_questions",
    question_text = """Consider the following survey question: {{draft_text}}
    Identify the problematic phrases in the excerpt and suggestion a revised version of it."""
)

Create "scenarios" of the question with the texts as paraemeters:

In [6]:
scenarios = [Scenario({"draft_text": text}) for text in draft_texts]

Select an LLM and administer the parameterized questions:

In [7]:
Model.available()

['claude-3-haiku-20240307',
 'claude-3-opus-20240229',
 'claude-3-sonnet-20240229',
 'dbrx-instruct',
 'gemini_pro',
 'gpt-3.5-turbo',
 'gpt-4-1106-preview',
 'llama-2-13b-chat-hf',
 'llama-2-70b-chat-hf',
 'mixtral-8x7B-instruct-v0.1']

In [8]:
model = Model('gpt-4-1106-preview', cache=False)

In [9]:
results = question.by(scenarios).by(agent).by(model).run()

In [10]:
results.columns

['agent.agent_name',
 'agent.background',
 'answer.review_questions',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.review_questions_system_prompt',
 'prompt.review_questions_user_prompt',
 'raw_model_response.review_questions_raw_model_response',
 'scenario.draft_text']

View select components of results:

In [11]:
(results
 .select("scenario.*", "answer.*")
 .print(pretty_labels={"scenario.draft_text":"Draft text", "answer.review_questions": "Evaluation"})
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Draft text                                             ┃ Evaluation                                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Do you believe our IT team's collaborative synergy     │ The phrase 'collaborative synergy' is jargon that may  │
│ effectively optimizes our digital infrastructure?      │ not be clear to all respondents, and 'effectively      │
│                                                        │ optimizes' is vague and could be interpreted in        │
│                                                        │ various ways. A revised version of the question could  │
│                                                        │ be: 'Do you believe our IT team works well together to │
│                                                        │ improve our digital systems?' This version uses more   │
│                                                        │ straightforward language and is likely to be           │
│                                                        │ understood by a wider audience.                        │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ Do you feel the product is almost always of good       │ The phrase 'almost always' is problematic as it        │
│ quality?                                               │ combines two qualifiers 'almost' and 'always' which    │
│                                                        │ can be confusing for respondents. The term 'good       │
│                                                        │ quality' is also vague and subjective. A revised       │
│                                                        │ version of the question could be: 'How would you rate  │
│                                                        │ the quality of the product?' with a clear scale        │
│                                                        │ provided, such as 'Excellent, Very Good, Good, Fair,   │
│                                                        │ Poor'.                                                 │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ On a scale of 1 to 5, where 1 means strongly agree and │ The problematic phrase in the survey question is the   │
│ 5 means strongly disagree, how satisfied are you with  │ scale description, where '1 means strongly agree and 5 │
│ our service?                                           │ means strongly disagree.' This does not align with the │
│                                                        │ question asked about satisfaction level. A revised     │
│                                                        │ version of the question could be: 'On a scale of 1 to  │
│                                                        │ 5, where 1 means very dissatisfied and 5 means very    │
│                                                        │ satisfied, how satisfied are you with our service?'    │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ What do you think of our recent implementation of      │ The question 'What do you think of our recent          │
│ Project X57?                                           │ implementation of Project X57?' is open-ended and      │
│                                                        │ lacks specificity, which could lead to a wide range of │
│                                                        │ responses that may be difficult to analyze             │
│                                                        │ quantitatively. Additionally, it assumes that          │
│                                                       

## Qualitative reviews
In this example we read in a set of hypothetical customer service tickets and prompt an LLM to extract a set of themes that we could use in follow-on questions (e.g., as a set of options to multiple choice questions).

In [12]:
from edsl.questions import QuestionList

In [13]:
tickets = [
    "I waited for 20 minutes past the estimated arrival time, and the driver still hasn't arrived. This made me late for my appointment.",
    "The driver was very rude and had an unpleasant attitude during the entire ride. It was an uncomfortable experience.",
    "The driver was speeding and frequently changing lanes without signaling. I felt unsafe throughout the ride.",
    "The car I rode in was dirty and messy. There were crumbs on the seats, and it didn't look like it had been cleaned in a while.",
    "The driver took a longer route, which resulted in a significantly higher fare than expected. I believe they intentionally extended the trip.",
    "I was charged for a ride that I did not take. The ride appears on my account, but I was not in the vehicle at that time.",
    "I left my wallet in the car during my last ride. I've tried contacting the driver, but I haven't received a response."
]

In [14]:
a_customer_service = Agent(traits = {"background": "You are an experienced customer service agent for a ridesharing company."})

In [15]:
q_topics = QuestionList(
    question_name = "ticket_topics",
    question_text = "Create a list of the topics raised in these customer service tickets: {{tickets_texts}}."
)

In [16]:
scenario = Scenario({"tickets_texts": "; ".join(tickets)})

In [17]:
topics = q_topics.by(scenario).by(a_customer_service).by(model).run()

In [18]:
topics.select("ticket_topics").to_list()[0]

['Late driver arrival',
 'Driver rudeness',
 'Unsafe driving',
 'Vehicle cleanliness',
 'Route taken and fare issue',
 'Incorrect charge for un-taken ride',
 'Lost item and driver unresponsiveness']

## Data labeling
In this example we prompt an LLM to rating the seriousness of tickets about safety issues.

See this notebook as well for a more complex data labeling exercise: <a href="https://deepnote.com/workspace/expected-parrot-c2fa2435-01e3-451d-ba12-9c36b3b87ad9/project/Expected-Parrot-examples-b457490b-fc5d-45e1-82a5-a66e1738a4b9/notebook/Data%20Labeling%20Agents-ed823c7d26d6410cb357d0b81ff95d80">Data Labeling Agents</a>.

In [19]:
from edsl.questions import QuestionLinearScale

In [20]:
safety_tickets = [
    "During my ride, I noticed that the driver was frequently checking their phone for directions, which made me a bit uncomfortable. It didn't feel like they were fully focused on the road.",
    "The driver had to brake abruptly to avoid a collision with another vehicle. It was a close call, and it left me feeling quite shaken. Please address this issue.",
    "I had a ride with a driver who was clearly speeding and weaving in and out of traffic. Their reckless driving put my safety at risk, and I'm very concerned about it.",
    "My ride was involved in a minor accident, and although no one was seriously injured, it was a scary experience. The driver is handling the situation, but I wanted to report it.",
    "I had a ride with a driver who exhibited aggressive and threatening behavior towards me during the trip. I felt genuinely unsafe and want this matter to be taken seriously."
]

In [21]:
q_rating = QuestionLinearScale(
    question_name = "safety_rating",
    question_text = """On a scale from 0-10 rate the seriousness of the issue raised in this customer service ticket
    (0 = Not serious, 10 = Extremely serious): {{ticket}}""",
    question_options = [0,1,2,3,4,5,6,7,8,9,10]
)

In [22]:
scenarios = [Scenario({"ticket": safety_ticket}) for safety_ticket in safety_tickets]

In [23]:
r_rating = q_rating.by(scenarios).by(a_customer_service).by(model).run()

In [24]:
(r_rating
 .select("scenario.*", "answer.*")
 .print()
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ scenario                                      ┃ answer                                         ┃ answer         ┃
┃ .ticket                                       ┃ .safety_rating_comment                         ┃ .safety_rating ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ I had a ride with a driver who exhibited      │ The safety of our customers is our top         │ 10             │
│ aggressive and threatening behavior towards   │ priority. Aggressive and threatening behavior  │                │
│ me during the trip. I felt genuinely unsafe   │ by a driver is unacceptable and is taken very  │                │
│ and want this matter to be taken seriously.   │ seriously. We will investigate this matter     │                │
│                                               │ immediately and take appropriate action.       │                │
├───────────────────────────────────────────────┼────────────────────────────────────────────────┼────────────────┤
│ I had a ride with a driver who was clearly    │ The issue described is extremely serious as it │ 10             │
│ speeding and weaving in and out of traffic.   │ involves reckless driving which puts the       │                │
│ Their reckless driving put my safety at risk, │ passenger's safety at risk. This is a          │                │
│ and I'm very concerned about it.              │ violation of our safety policies and will be   │                │
│                                               │ addressed immediately.                         │                │
├───────────────────────────────────────────────┼────────────────────────────────────────────────┼────────────────┤
│ During my ride, I noticed that the driver was │ Safety is a top priority, and a driver not     │ 7              │
│ frequently checking their phone for           │ fully focused on the road due to phone usage   │                │
│ directions, which made me a bit               │ for navigation or any other reason is a        │                │
│ uncomfortable. It didn't feel like they were  │ serious concern. It warrants immediate         │                │
│ fully focused on the road.                    │ attention and corrective action to ensure the  │                │
│                                               │ safety of passengers and adherence to safe     │                │
│                                               │ driving practices.                             │                │
├───────────────────────────────────────────────┼────────────────────────────────────────────────┼────────────────┤
│ My ride was involved in a minor accident, and │ While no one was seriously injured, any        │ 8              │
│ although no one was seriously injured, it was │ accident involving a rideshare vehicle is a    │                │
│ a scary experience. The driver is handling    │ high-priority concern that must be thoroughly  │                │
│ the situation, but I wanted to report it.     │ investigated for safety and insurance          │                │
│                                               │ purposes.                                      │                │
├───────────────────────────────────────────────┼────────────────────────────────────────────────┼────────────────┤
│ The driver had to brake abruptly to avoid a   │ The safety and comfort of our passengers is of │ 8              │
│ collision with another vehicle. It was a      │ utmost importance. An incident where a driver  │                │
│ close call, and it left me feeling quite      │ has to brake abruptly to avoid a collision is  │                │
│ shaken. Please address this issue.            │ a serious safety concern and can be very       │                │
│                                               │ distre

## Creating new methods
We can use the question prompts to create new methods, such as a translator:

In [25]:
def translate_to_german(text):
    q = QuestionFreeText(
        question_name = "deutsch",
        question_text = "Please translate '{{ text }}' into German"
    )
    result = q.by(Scenario({'text':text})).run()
    return result.select("deutsch").print()

In [26]:
translate_to_german("Hello, friend, have you been traveling?")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                          ┃
┃ .deutsch                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Hallo, Freund, bist du gereist? │
└─────────────────────────────────┘

---
<p style="font-size: 14px;">Copyright © 2024 Expected Parrot, Inc. All rights reserved.   <a href="www.expectedparrot.com" style="color:#130061">www.expectedparrot.com</a></p>